<a href="https://colab.research.google.com/github/thisisreallife/Medium/blob/master/AA_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
%reset -f
import numpy as np
import pandas as pd 
import scipy.stats as stats
import matplotlib.pyplot as plt
import tqdm
from multiprocessing import Pool
from itertools import product
import plotly.express as px
def unpack_args(func):
    from functools import wraps
    @wraps(func)
    def wrapper(args):
        if isinstance(args, dict):
            return func(**args)
        else:
            return func(*args)
    return wrapper

In [81]:
def power_compare(sample_size, effect_size, M_for_simulation):
    p_test_res = []
    t_test_res = []
    for i in range(M_for_simulation):
        t_stats = stats.norm.rvs(effect_size,1,sample_size)
        p_values = 2*(1-stats.norm.cdf(abs(t_stats)))
        # kstest p_values against unifrom distribution
        p_test_res.append(stats.kstest(p_values, lambda x: stats.uniform.cdf(x))[1])
        # kstest t_stats against t distribution
        t_test_res.append(stats.kstest(t_stats, lambda x: stats.norm.cdf(x))[1])
    return (np.array(p_test_res)<0.05).mean(), (np.array(t_test_res)<0.05).mean()
    # return np.array(p_test_res), np.array(t_test_res)

In [82]:
power_compare(30, 0.1,10000)

(0.0457, 0.0739)

In [83]:
power_compare(300, 0.1,10000)

(0.0481, 0.328)

In [84]:
power_compare(3000, 0.1, 10000)

(0.058, 0.998)

In [85]:
@unpack_args
def power_compare(sample_size, effect_size, M_for_simulation):
    p_test_res = []
    t_test_res = []
    for i in range(M_for_simulation):
        t_stats = stats.norm.rvs(effect_size,1,sample_size)
        p_values = 2*(1-stats.norm.cdf(abs(t_stats)))
        # kstest p_values against unifrom distribution
        p_test_res.append(stats.kstest(p_values, lambda x: stats.uniform.cdf(x))[1])
        # kstest t_stats against t distribution
        t_test_res.append(stats.kstest(t_stats, lambda x: stats.norm.cdf(x))[1])
    return (np.array(p_test_res)<0.05).mean(), (np.array(t_test_res)<0.05).mean()
    # return np.array(p_test_res), np.array(t_test_res)

In [86]:
sample_size = [30, 100, 300] 
effect_size = np.linspace(0.1,1,10)
sim_M = [10000]
para_sim = list(product(sample_size,effect_size, sim_M)) # Zip params to use in pool.imap

In [87]:
results = []
with Pool() as pool:
    for result in tqdm.tqdm(pool.imap(power_compare, para_sim), total=para_sim.__len__()):
        results.append(result)

100%|██████████| 30/30 [05:47<00:00, 11.57s/it]


In [90]:
output = pd.concat([pd.DataFrame(para_sim), pd.DataFrame(results)], axis = 1)
output.columns = ['sample_size', 'effect_size', 'sim_M', 'p_values_power', 't_stats_power']

In [91]:
output.head()

,sample_size,effect_size,sim_M,p_values_power,t_stats_power
0,30,0.1,10000,0.0508,0.0756
1,30,0.2,10000,0.0517,0.1562
2,30,0.3,10000,0.0598,0.2909
3,30,0.4,10000,0.0750,0.4721
4,30,0.5,10000,0.1032,0.6433


In [106]:
df_plot = pd.melt(output,id_vars = ['sample_size', 'effect_size'], value_vars=['p_values_power', 't_stats_power'], value_name = 'power',var_name = 'type')

In [137]:
df_plot.head()
df_plot['type'] = df_plot['type'].map({'p_values_power':'p_values_based', 't_stats_power':'t_stats_based'})

Conclusion:
1. Comparing to p_values_base testing, t_stats_based testing has a higher power.
2. When effect size is small(<0.1), we need more than 300 samples in t_stats_based testing to achieve 80% power

In [139]:
fig = px.line(data_frame = df_plot, x = 'effect_size', y = 'power', color = 'type', facet_col = 'sample_size', title = 'Power Analysis Under DIfferent Sample Size')
fig.update_annotations(font=dict(size=16))
fig.update_yaxes(matches=None, showticklabels=True)
fig.update_layout(
    # legend_grouptitlefont_size = 16,
    # legend_title_font_color="green"
)
# fig.update_xaxes(title_font_family="Arial")
fig.add_hline(y=0.8, line_width=3, line_dash="dash", line_color="grey", annotation_text="Achieve 80% Power", annotation_position="top left")